In [ ]:
%cd ../airflow
%pwd

In [ ]:
from include import RawExtractor

extractor = RawExtractor(
    base_url="https://api.github.com/repos/henriquepgomide/caRtola/contents/data",
    path="./data/raw",
)

extractor.extract_static_files("2017")
extractor.extract_dynamic_files("2017")

In [ ]:
import pandas as pd
import uuid

In [ ]:
clubes_df = pd.read_csv("./data/raw/2017/times_ids/1.csv")
clubes_df = clubes_df[["id", "cod.2017", "nome.cbf"]].rename(
    columns={"cod.2017": "olderID", "id": "clubeID", "nome.cbf": "nome"}
)

partidas_df = pd.read_csv("./data/raw/2017/partidas/1.csv")

partidas_df = partidas_df.merge(
    right=clubes_df,
    left_on="home_team",
    right_on="nome",
    suffixes=(None, "_mandante"),
)

partidas_df = partidas_df.merge(
    right=clubes_df,
    left_on="away_team",
    right_on="nome",
    suffixes=(None, "_visitante"),
)

partidas_df["mandantePlacar"] = partidas_df["score"].str.split(" x ").str[0].astype(int)
partidas_df["visitantePlacar"] = partidas_df["score"].str.split(" x ").str[-1].astype(int)

def get_resultado(row):
    if row["mandantePlacar"] > row["visitantePlacar"]:
        return "Casa"
    elif row["mandantePlacar"] < row["visitantePlacar"]:
        return "Visitante"
    else:
        return "Empate"
    
partidas_df["resultado"] = partidas_df.apply(get_resultado, axis=1)
partidas_df["partidaID"] = [uuid.uuid4().hex for _ in range(len(partidas_df))]
    
partidas_df["temporada"] = 2017
partidas_df = partidas_df.rename(
    columns={
        "clubeID": "clubeMandanteID",
        "clubeID_visitante": "clubeVisitanteID",
        "round": "rodada",
    }
)

partidas_df = partidas_df[[
    "partidaID",
    "clubeMandanteID",
    "clubeVisitanteID",
    "mandantePlacar",
    "visitantePlacar",
    "rodada",
    "resultado",
    "temporada",
]]

partidas_df

In [ ]:
clubes_df = pd.read_csv("./data/raw/2017/times_ids/1.csv")
clubes_df = clubes_df[["id", "cod.2017", "nome.cartola"]].rename(
    columns={"cod.2017": "olderID", "id": "clubeID", "nome.cartola": "nome"}
)

scouts_df = pd.read_csv("./data/raw/2017/scouts_raw/1.csv")

scouts_df = scouts_df.merge(
    right=clubes_df,
    left_on="atletas.clube.id.full.name",
    right_on="nome",
)

scouts_df = scouts_df.rename(
    columns={
        'atletas.atleta_id': "atletaID",
        'Rodada': "rodada",
        'atletas.pontos_num': "pontos",
        'atletas.media_num': "pontosMedia",
        'atletas.preco_num': "preco", 
        'atletas.variacao_num': "precoVariacao",
    },
)

house_by_rounds = partidas_df[["partidaID", "rodada", "clubeMandanteID"]].rename(
    columns={"clubeMandanteID": "clubeID"}
)
visitor_by_rounds = partidas_df[["partidaID", "rodada", "clubeVisitanteID"]].rename(
    columns={"clubeVisitanteID": "clubeID"}
)
club_by_rounds = pd.concat([house_by_rounds, visitor_by_rounds])

scouts_df = scouts_df.merge(
    right=club_by_rounds,
    on=["rodada", "clubeID"],
)

scouts_df = scouts_df[[
    "clubeID",
    "partidaID",
    "atletaID",
    "pontos",
    "pontosMedia",
    "preco",
    "precoVariacao",
    "FS",
    "PE",
    "A",
    "FT",
    "FD",
    "FF",
    "G",
    "I",
    "PP",
    "RB",
    "FC",
    "GC",
    "CA",
    "CV",
    "SG",
    "DD",
    "DP",
    "GS",
]]

scouts_df

In [ ]:
clubes_df = pd.read_csv("./data/raw/2017/times_ids/1.csv")
clubes_df = clubes_df[["id", "cod.2017"]].rename(
    columns={"cod.2017": "olderID", "id": "clubeID"}
).drop_duplicates()

atletas_df = pd.read_csv("./data/raw/2017/jogadores/1.csv")

atletas_df = atletas_df.merge(
    right=clubes_df,
    left_on="ClubeID",
    right_on="olderID",
)

atletas_df.drop_duplicates(inplace=True)
atletas_df.rename(
    columns={
        "AtletaID": "atletaID",
        "Apelido": "apelido",
        "PosicaoID": "posicaoID",
    },
    inplace=True
)

atletas_df = atletas_df[[
    "atletaID",
    "clubeID",
    "posicaoID",
    "apelido",
]]

atletas_df

In [ ]:
posicoes_df = pd.read_csv("./data/raw/2017/posicoes_ids/1.csv")
posicoes_df.rename(
    columns={
        "Cod": "posicaoID",
        "Position": "nome",
        "abbr": "abreviacao",
    },
    inplace=True,
)

posicoes_df = posicoes_df[[
    "posicaoID",
    "nome",
    "abreviacao",
]]

posicoes_df

In [ ]:
clubes_df = pd.read_csv("./data/raw/2017/times_ids/1.csv")
clubes_df.rename(
    columns={
        "id": "clubeID",
        "nome.cbf": "nome",
    },
    inplace=True,
)

clubes_df = clubes_df[["clubeID", "nome", "abreviacao"]]
clubes_df.drop_duplicates(subset=["clubeID"], keep="last", inplace=True)

clubes_df